# About: Docker - Ready! on Ubuntu and Set! Go!

---

Install and configure Docker Engine and Docker Compose onto Ubuntu 16.04. Then, make sure "docker run" and "docker-compose up" work.

Ubuntu 16.04環境にDockerをインストールするためのNotebook。<br>
このNotebookによりインストールできるソフトウェアは以下の通り。

- Docker CE(Docker Engine)
- Docker Compose

Docker Engineのインストール手順は、[Get Docker CE for Ubuntu](https://docs.docker.com/engine/installation/linux/docker-ce/ubuntu/) (*2017/11/7時点での最新版*) を参考に作成している。また、Docker Composeのインストール手順は、[Install Docker Compose](https://docs.docker.com/compose/install/) (*2017/11/7時点での最新版*) を参考に作成している。

## *Operation Note*

*This is a cell for your own recording.  ここに経緯を記述*

## 設定情報

このNotebookで行う設定は、以下のようにする。

- NIIのベアメタルマシンを想定 ... Docker関係のディレクトリは `/mnt`　(Ephemeralなパーティション) に配置する
- クラウド運用チームでの利用を想定 ... Dockerのプライベートレジストリを使用するので、プライベートレジストリのホスト情報を明示する

する。

docker_optsの定義方法は[DAEMON CONFIGURATION FILE](https://docs.docker.com/engine/reference/commandline/dockerd/#daemon-configuration-file)を参照。

In [1]:
docker_tmp = "/mnt/docker-tmp"
docker_base = "/mnt/docker"
docker_opts ={"data-root": docker_base, "insecure-registries": ["XXX.XXX.XXX.93:5000"]}


# Notebookと環境のBinding

Inventory中のgroup名でBind対象ホスト(Docker Engineをインストールしたいホスト)を指示する。

In [2]:
target_group = 'test-vm'

Bind対象への疎通状態を確認する。

In [3]:
!ansible -m ping {target_group}

XXX.XXX.XXX.66 | SUCCESS => {
    "changed": false, 
    "failed": false, 
    "ping": "pong"
}


# Binding対象の確認

[Prerequisites](https://docs.docker.com/engine/installation/linux/ubuntulinux/#prerequisites)に示されているとおり、このNotebookを使ってDockerをインストールする対象のホストは、以下の条件を満たしている必要がある。

もし、このインストール手順に失敗したら、**この条件を満たす状態にマシンを戻す(パッケージの削除, マシンの再プロビジョニングなど)**ことで、**(このNotebookによって)Dockerをインストール可能な状態に戻す**ことができる。

## 64bit版を使う

Ubuntuのバージョンにかかわらず、Docker Engineを動作させるには64bit版が必要。

In [4]:
!ansible -a 'uname -m' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
x86_64



## kernel versionは最低3.10

3.10未満の古いバージョンの場合はDockerの機能の一部が使えなかったり、データロストやpanicを生じる可能性がある。

そのため、以下のバージョン表示が3.10以上であることを確認しておく。

In [5]:
!ansible -a 'uname -r' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
4.4.0-97-generic



## Ubuntuは16.04を想定

加えて、このNotebookは、**Ubuntu 16.04がインストールされた環境にBindingされることを前提として実装**している。

以下のコマンドの出力が Ubuntu 16.04 であることを確認する。

> 16.04以外のUbuntuの場合はRepositoryのURLなどを適宜読み替えること

In [6]:
!ansible -a 'lsb_release -a' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.3 LTS
Release:	16.04
Codename:	xenialNo LSB modules are available.



## Ubuntu-maintainedなDockerがインストールされていないこと

Ubuntu-maintainedなDocker(`docker-io` パッケージ)がすでにインストールされているとファイル構成など競合するかもしれない。そのため、念のため以下のコマンドに**失敗する(FAILED)** ことを確認しておく。

In [7]:
!ansible -b -m shell -a 'dpkg -l | grep docker.io' {target_group}

XXX.XXX.XXX.66 | FAILED | rc=1 >>
non-zero return code



RuntimeError: Unexpected exit code: 2

## 古いrepositoryからインストールされていないこと

古いrepositoryでは `lxc-docker` というパッケージ名だった時代があった・・・これがインストールされていないことも念のため確認しておく。以下のコマンドに**失敗する(FAILED)** ことを確認しておく。

In [8]:
!ansible -b -m shell -a 'dpkg -l | grep lxc-docker' {target_group}

XXX.XXX.XXX.66 | FAILED | rc=1 >>
non-zero return code



RuntimeError: Unexpected exit code: 2

## APTにdocker-engineのrepositoryが未登録

このNotebook適用時は、docker-engineのrepositoryが未登録であることを前提としている。すでにrepositoryが登録されている場合、このNotebookで指定したパッケージが適切にインストールされないかもしれない。念のため以下のコマンドの実行結果に、**何もインストール候補バージョンが現れない**ことを確認しておく。

In [9]:
!ansible -b -m shell -a 'apt-get update && apt-cache policy docker-ce' {target_group}

 [WARNING]: Consider using apt module rather than running apt-get

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Hit:1 http://us.archive.ubuntu.com/ubuntu xenial InRelease
Get:2 http://security.ubuntu.com/ubuntu xenial-security InRelease [102 kB]
Get:3 http://us.archive.ubuntu.com/ubuntu xenial-updates InRelease [102 kB]
Get:4 http://us.archive.ubuntu.com/ubuntu xenial-backports InRelease [102 kB]
Get:5 http://us.archive.ubuntu.com/ubuntu xenial-updates/main amd64 Packages [652 kB]
Get:6 http://security.ubuntu.com/ubuntu xenial-security/main amd64 Packages [381 kB]
Get:7 http://us.archive.ubuntu.com/ubuntu xenial-updates/main i386 Packages [617 kB]
Get:8 http://security.ubuntu.com/ubuntu xenial-security/main i386 Packages [352 kB]
Get:9 http://security.ubuntu.com/ubuntu xenial-security/main Translation-en [168 kB]
Get:10 http://us.archive.ubuntu.com/ubuntu xenial-updates/main Translation-en [273 kB]
Get:11 http://security.ubuntu.com/ubuntu xenial-security/restricted amd64 Packages [7,472 B]
Get:1

# Docker Engineのインストール

Bind対象にDocker Engineをインストールする。

## apt sourcesの更新

Docker社のrepositoryの情報をBind対象マシンに追加する。

まず、HTTPSのrepositoryからパッケージをインストールできるようにしておく。

In [10]:
!ansible -b -m shell -a 'apt-get update && \
                         apt-get install -y apt-transport-https ca-certificates curl' {target_group}

 [WARNING]: Consider using apt module rather than running apt-get

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Hit:1 http://us.archive.ubuntu.com/ubuntu xenial InRelease
Hit:2 http://security.ubuntu.com/ubuntu xenial-security InRelease
Hit:3 http://us.archive.ubuntu.com/ubuntu xenial-updates InRelease
Hit:4 http://us.archive.ubuntu.com/ubuntu xenial-backports InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
ca-certificates is already the newest version (20170717~16.04.1).
ca-certificates set to manually installed.
The following additional packages will be installed:
  libasn1-8-heimdal libcurl3-gnutls libgssapi3-heimdal libhcrypto4-heimdal
  libheimbase1-heimdal libheimntlm0-heimdal libhx509-5-heimdal
  libkrb5-26-heimdal libldap-2.4-2 libroken18-heimdal librtmp1 libsasl2-2
  libsasl2-modules libsasl2-modules-db libwind0-heimdal
Suggested packages:
  libsasl2-modules-otp libsasl2-modules-ldap libsasl2-modules-sql
  libsasl2-mod

GPG keyを追加しておく。

In [11]:
!ansible -b -m shell \
         -a 'curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -' {target_group}

 [WARNING]: Consider using get_url or uri module rather than running curl

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
OK



以下のコマンドで現れるfingerprintが `9DC8 5822 9FC7 DD38 854A E2D8 8D81 803C 0EBF CD88` であることを確認しておく。

In [12]:
!ansible -b -a 'apt-key fingerprint 0EBFCD88' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
pub   4096R/0EBFCD88 2017-02-22
      Key fingerprint = 9DC8 5822 9FC7 DD38 854A  E2D8 8D81 803C 0EBF CD88
uid                  Docker Release (CE deb) <docker@docker.com>
sub   4096R/F273FCD8 2017-02-22



Ubuntuのバージョンに応じたrepositoryのURLを追加する。このNotebookでは **16.04(Xenial)** を追加。

In [13]:
!ansible -b -m shell -a 'echo deb [arch=amd64] https://download.docker.com/linux/ubuntu \
                              xenial stable \
                              > /etc/apt/sources.list.d/docker.list' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>




aptがrepositoryから情報を取得できているかの確認。

In [14]:
!ansible -b -m shell -a 'apt-get update && \
                         apt-cache policy docker-ce' {target_group}

 [WARNING]: Consider using apt module rather than running apt-get

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Get:1 https://download.docker.com/linux/ubuntu xenial InRelease [49.8 kB]
Hit:2 http://us.archive.ubuntu.com/ubuntu xenial InRelease
Hit:3 http://security.ubuntu.com/ubuntu xenial-security InRelease
Get:4 https://download.docker.com/linux/ubuntu xenial/stable amd64 Packages [2,579 B]
Hit:5 http://us.archive.ubuntu.com/ubuntu xenial-updates InRelease
Hit:6 http://us.archive.ubuntu.com/ubuntu xenial-backports InRelease
Fetched 52.4 kB in 0s (59.8 kB/s)
Reading package lists...
docker-ce:
  Installed: (none)
  Candidate: 17.09.0~ce-0~ubuntu
  Version table:
     17.09.0~ce-0~ubuntu 500
        500 https://download.docker.com/linux/ubuntu xenial/stable amd64 Packages
     17.06.2~ce-0~ubuntu 500
        500 https://download.docker.com/linux/ubuntu xenial/stable amd64 Packages
     17.06.1~ce-0~ubuntu 500
        500 https://download.docker.com/linux/ubuntu xenial/stable amd64 Packages
    

## パッケージのインストール

`docker-ce` パッケージをインストールする。

In [15]:
!ansible -b -m shell -a 'apt-get update && \
                         apt-get install -y docker-ce' {target_group}

 [WARNING]: Consider using apt module rather than running apt-get

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Hit:1 https://download.docker.com/linux/ubuntu xenial InRelease
Hit:2 http://us.archive.ubuntu.com/ubuntu xenial InRelease
Hit:3 http://security.ubuntu.com/ubuntu xenial-security InRelease
Hit:4 http://us.archive.ubuntu.com/ubuntu xenial-updates InRelease
Hit:5 http://us.archive.ubuntu.com/ubuntu xenial-backports InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  apparmor aufs-tools cgroupfs-mount git git-man iptables libapparmor-perl
  liberror-perl libgdbm3 libltdl7 libnfnetlink0 libperl5.22 patch perl
  perl-modules-5.22 rename rsync xz-utils
Suggested packages:
  apparmor-profiles apparmor-profiles-extra apparmor-docs apparmor-utils
  mountall git-daemon-run | git-daemon-sysvinit git-doc git-el git-email
  git-gui gitk gitweb git-arch git-cvs git-mediawiki git-sv

## Docker Engineの設定変更

あらかじめ定義した設定情報にしたがい、Docker Engineに与えるDefault Configを指定する。

In [16]:
import tempfile
temp_dir = tempfile.mkdtemp()
temp_dir

'/tmp/tmpkLyFWr'

In [17]:
import os
import json
with open(os.path.join(temp_dir, 'daemon.json'), 'w') as f:
    f.write(json.dumps(docker_opts))
!cat {temp_dir}/daemon.json

{"data-root": "/mnt/docker", "insecure-registries": []}

In [19]:
import os
with open(os.path.join(temp_dir, 'tmpdir.conf'), 'w') as f:
    f.write('''# Systemd drop-in configuration for Docker
[Service]
Environment="DOCKER_TMPDIR={docker_tmp}"'''.format(docker_tmp=docker_tmp))
!cat {temp_dir}/tmpdir.conf

# Systemd drop-in configuration for Docker
[Service]
Environment="DOCKER_TMPDIR=/mnt/docker-tmp"

ローカルに作った configファイル を、Bind対象の/etc/default/dockerにコピーし、Docker Engineに反映する。

In [20]:
!ansible -b -m copy -a 'src={temp_dir}/daemon.json dest=/etc/docker/daemon.json' {target_group}
!ansible -b -m file -a 'path=/etc/systemd/system/docker.service.d state=directory' {target_group}
!ansible -b -m copy -a 'src={temp_dir}/tmpdir.conf dest=/etc/systemd/system/docker.service.d/tmpdir.conf' {target_group}

!ansible -b -m file -a 'path={docker_tmp} state=directory' {target_group}
!ansible -b -a 'systemctl daemon-reload' {target_group}
!ansible -b -m service -a 'name=docker state=restarted' {target_group}

XXX.XXX.XXX.66 | SUCCESS => {
    "changed": true, 
    "checksum": "d7af45d32223f16ef2c90c84519f37e8bd425777", 
    "dest": "/etc/docker/daemon.json", 
    "failed": false, 
    "gid": 0, 
    "group": "root", 
    "md5sum": "d24bc7e71c6f5bb90f4c60847e086406", 
    "mode": "0644", 
    "owner": "root", 
    "size": 55, 
    "src": "/home/ansible/.ansible/tmp/ansible-tmp-1510008598.03-253168517244957/source", 
    "state": "file", 
    "uid": 0
}
XXX.XXX.XXX.66 | SUCCESS => {
    "changed": true, 
    "failed": false, 
    "gid": 0, 
    "group": "root", 
    "mode": "0755", 
    "owner": "root", 
    "path": "/etc/systemd/system/docker.service.d", 
    "size": 6, 
    "state": "directory", 
    "uid": 0
}
XXX.XXX.XXX.66 | SUCCESS => {
    "changed": true, 
    "checksum": "17e2db6d5b09b4a11d6ad962ea1b86da7d2c553b", 
    "dest": "/etc/systemd/system/docker.service.d/tmpdir.conf", 
    "failed": false, 
    "gid": 0, 
    "group": "root", 
    "md5sum": "2aea202d5e2f6713a286fb9f187cd3dd

念のため、Docker Engineにより /mnt/docker, /mnt/docker-tmp にファイルが作成されていることを確認する。

In [21]:
!ansible -b -a 'ls -la {docker_tmp} {docker_base}' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
/mnt/docker:
total 0
drwx--x--x 11 root root 139 Nov  7 07:50 .
drwxr-xr-x  4 root root  38 Nov  7 07:50 ..
drwx------  2 root root  24 Nov  7 07:50 builder
drwx------  2 root root   6 Nov  7 07:50 containers
drwx------  3 root root  22 Nov  7 07:50 image
drwxr-x---  3 root root  19 Nov  7 07:50 network
drwx------  3 root root  40 Nov  7 07:50 overlay2
drwx------  4 root root  32 Nov  7 07:50 plugins
drwx------  2 root root   6 Nov  7 07:50 swarm
drwx------  2 root root   6 Nov  7 07:50 trust
drwx------  2 root root  25 Nov  7 07:50 volumes

/mnt/docker-tmp:
total 0
drwxr-xr-x 2 root root  6 Nov  7 07:50 .
drwxr-xr-x 4 root root 38 Nov  7 07:50 ..



In [22]:
!ansible -b -a 'systemctl status docker' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
● docker.service - Docker Application Container Engine
   Loaded: loaded (/lib/systemd/system/docker.service; enabled; vendor preset: enabled)
  Drop-In: /etc/systemd/system/docker.service.d
           └─tmpdir.conf
   Active: active (running) since Tue 2017-11-07 07:50:08 JST; 12s ago
     Docs: https://docs.docker.com
 Main PID: 5046 (dockerd)
    Tasks: 23
   Memory: 23.1M
      CPU: 207ms
   CGroup: /system.slice/docker.service
           ├─5046 /usr/bin/dockerd -H fd://
           └─5057 docker-containerd -l unix:///var/run/docker/libcontainerd/docker-containerd.sock --metrics-interval=0 --start-timeout 2m --state-dir /var/run/docker/libcontainerd/containerd --shim docker-containerd-shim --runtime docker-runc

Nov 07 07:50:08 ubuntu dockerd[5046]: time="2017-11-07T07:50:08.040890717+09:00" level=warning msg="Your kernel does not support swap memory limit"
Nov 07 07:50:08 ubuntu dockerd[5046]: time="2017-11-07T07:50:08.041074723+09:00" level=warni

Docker Engineのバージョンを確認する。

In [23]:
!ansible -b -a 'docker version' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Client:
 Version:      17.09.0-ce
 API version:  1.32
 Go version:   go1.8.3
 Git commit:   afdb6d4
 Built:        Tue Sep 26 22:42:18 2017
 OS/Arch:      linux/amd64

Server:
 Version:      17.09.0-ce
 API version:  1.32 (minimum version 1.12)
 Go version:   go1.8.3
 Git commit:   afdb6d4
 Built:        Tue Sep 26 22:40:56 2017
 OS/Arch:      linux/amd64
 Experimental: false



Docker Engineの設定状況も確認しておく。

In [24]:
!ansible -b -a 'docker info' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Containers: 0
 Running: 0
 Paused: 0
 Stopped: 0
Images: 0
Server Version: 17.09.0-ce
Storage Driver: overlay2
 Backing Filesystem: xfs
 Supports d_type: true
 Native Overlay Diff: true
Logging Driver: json-file
Cgroup Driver: cgroupfs
Plugins:
 Volume: local
 Network: bridge host macvlan null overlay
 Log: awslogs fluentd gcplogs gelf journald json-file logentries splunk syslog
Swarm: inactive
Runtimes: runc
Default Runtime: runc
Init Binary: docker-init
containerd version: 06b9cb35161009dcb7123345749fef02f7cea8e0
runc version: 3f2f8b84a77f73d38244dd690525642a72156c64
init version: 949e6fa
Security Options:
 apparmor
 seccomp
  Profile: default
Kernel Version: 4.4.0-97-generic
Operating System: Ubuntu 16.04.3 LTS
OSType: linux
Architecture: x86_64
CPUs: 6
Total Memory: 23.55GiB
Name: ubuntu
ID: J5HS:7GY5:4GOE:7DYW:S3U3:UOLS:YDCL:SJCW:DVR2:KRW4:O3FA:NV5P
Docker Root Dir: /mnt/docker
Debug Mode (client): false
Debug Mode (server): false
Registry: https

# Docker Composeのインストール

*2017/11/7時点* では、docker-composeのバージョンは1.17.0となる。

In [25]:
!ansible -b -m shell \
         -a 'curl -L https://github.com/docker/compose/releases/download/1.17.0/docker-compose-`uname -s`-`uname -m` \
                 > /usr/local/bin/docker-compose' {target_group}

 [WARNING]: Consider using get_url or uri module rather than running curl

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   617    0   617    0     0    702      0 --:--:-- --:--:-- --:--:--   701
100 8649k  100 8649k    0     0  1128k      0  0:00:07  0:00:07 --:--:-- 1920k



In [26]:
!ansible -b -a 'chmod +x /usr/local/bin/docker-compose' {target_group}

 [WARNING]: Consider using file module with mode rather than running chmod

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>




In [27]:
!ansible -b -a 'docker-compose --version' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
docker-compose version 1.17.0, build ac53b73



# Docker Engineの動作確認

まずはお試しで、hello-worldイメージを実行してみる。`Hello from Docker`のようなメッセージが表示されたらOK。

In [28]:
!ansible -b -a 'docker run hello-world' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://cloud.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/engine/userguide/Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
9a0669468bf7: Pulling fs layer
9a0669468bf7: Download complete
9a0669468bf

Dockerのhello-worldイメージが実行された。OK。

# Docker Composeの動作確認

Docker Composeが動作することも確認しておく。

まずローカルにdocker-compose.ymlファイルを準備。

In [29]:
!mkdir -p {temp_dir}/hello-compose/

In [30]:
%%writefile {temp_dir}/hello-compose/docker-compose.yml
version: '2'
services:
  test-hello-world:
    image: hello-world

Writing /tmp/tmpkLyFWr/hello-compose/docker-compose.yml


作成したdocker-compose.ymlを、Bind対象ホストにアップロードする。

In [31]:
!ansible -b -m copy -a 'src={temp_dir}/hello-compose dest=~' {target_group}

XXX.XXX.XXX.66 | SUCCESS => {
    "changed": true, 
    "checksum": "19f3b6e911e6461b09d1ddaf5a5f7f8dbc538a6e", 
    "dest": "/root/hello-compose/docker-compose.yml", 
    "failed": false, 
    "gid": 0, 
    "group": "root", 
    "md5sum": "a157d75e4107e5c1aa63441b9ff90758", 
    "mode": "0644", 
    "owner": "root", 
    "size": 65, 
    "src": "/home/ansible/.ansible/tmp/ansible-tmp-1510008665.9-219846645213383/source", 
    "state": "file", 
    "uid": 0
}


実行してみる。`Hello from Docker`のようなメッセージが表示されたらOK。

In [32]:
!ansible -b -a 'chdir=~/hello-compose docker-compose up' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Attaching to hellocompose_test-hello-world_1
test-hello-world_1  | 
test-hello-world_1  | Hello from Docker!
test-hello-world_1  | This message shows that your installation appears to be working correctly.
test-hello-world_1  | 
test-hello-world_1  | To generate this message, Docker took the following steps:
test-hello-world_1  |  1. The Docker client contacted the Docker daemon.
test-hello-world_1  |  2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
test-hello-world_1  |  3. The Docker daemon created a new container from that image which runs the
test-hello-world_1  |     executable that produces the output you are currently reading.
test-hello-world_1  |  4. The Docker daemon streamed that output to the Docker client, which sent it
test-hello-world_1  |     to your terminal.
test-hello-world_1  | 
test-hello-world_1  | To try something more ambitious, you can run an Ubuntu container with:
test-hello-world_1  |  $ docker run -

# 後始末

一時ディレクトリを削除する。

In [33]:
!rm -fr {temp_dir}